In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from konlpy.tag import Okt

import os

from keybert import KeyBERT
import tqdm

c:\ProgramData\Anaconda3\envs\DINS\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [3]:
data = pd.read_csv('./Data/SBS.csv')

data.head(1)

,title,headline,date,link,content,category,site
0,"BTS ""웰컴 제네레이션…변화에 겁먹지 않고 새로운 길로""","미국 뉴욕을 찾은 방탄소년단, BTS가 ""지금 청년들은 변화에 겁먹기보단 '웰컴'이...",2021.09.20,https://news.sbs.co.kr//news/endPage.do?news_i...,"미국 뉴욕을 찾은 방탄소년단, BTS가 ""지금 청년들은 변화에 겁먹기보단 '웰컴...",국제,sbs


In [45]:
# date 변경, link col_name 변경 및 저장

# data.rename(columns={'news_link' : 'link'}, inplace=True)

# data.date = data.date.str.split(' ').str[0]

# data.head(1)

# data.to_csv('./Data/SBS.csv', index=False)

In [26]:
# okt = Okt()

# contents_nouns = []

In [6]:
# contents_nouns.append(p.map(get_nouns, contents))

In [4]:
# for text in contents:
#     nouns = okt.nouns(text)
    
#     text_nouns = ''
#     for noun in nouns:
#         text_nouns += noun + ' '
    
#     contents_nouns.append(text_nouns)
#     print(text_nouns)

In [22]:
# contents = data.iloc[79573:].content

# len(contents)

252

In [4]:
data_sep = data.iloc[78237:]

len(data_sep)

1588

In [5]:
dates = data_sep.date.unique()
# '2022.09.13', '2022.09.14', '2022.09.15', '2022.09.16',
# '2022.09.17', '2022.09.18', '2022.09.19'

In [17]:
categories = data_sep.category.unique()
# '사회', '국제', '스포츠', '생활·문화', '경제', '정치', '연예'

In [11]:
data_date = []

for date in dates:
    data_date.append(data_sep.groupby('date').get_group(date))

In [23]:
datas = []


for data in data_date:
    data_category = []
    for category in categories:
        data_category.append(data.groupby('category').get_group(category).content)
    
    datas.append(data_category)

In [41]:
okt = Okt()

contents_nouns = []

In [42]:
datas_nouns = []

for data_date in datas:
    contents_nouns = []
    
    for contents in data_date:
        content_nouns = ''
        
        for text in contents:
            text_nouns = okt.nouns(text)
        
            for noun in text_nouns:
                content_nouns += noun + ' '
            
        contents_nouns.append(content_nouns)
        
    datas_nouns.append(contents_nouns)

In [25]:
# content_nouns = ''

# for text in contents:
#     text_nouns = okt.nouns(text)
    
#     for noun in text_nouns:
#         content_nouns += noun + ' '
    

# contents_nouns.append(content_nouns)

In [55]:
model = KeyBERT('distilbert-base-nli-mean-tokens')

In [56]:
datas_keywords = []

for data_date in datas_nouns:
    contents_keywords = []
    
    for content_nouns in data_date:
        contents_keywords.append(model.extract_keywords(content_nouns, keyphrase_ngram_range=(1,1), top_n=20, use_mmr=True))
    
    datas_keywords.append(contents_keywords)

In [65]:
result_keywords = []

date_idx = 0
category_idx = 0

for data_keywords in datas_keywords:
    for keywords in data_keywords:
        for keyword in keywords:
            result_keywords.append((dates[date_idx], categories[category_idx], ) + keyword) # (date, category, keyword, importance)
    
        category_idx += 1
            
    date_idx += 1
    category_idx = 0

In [68]:
result = pd.DataFrame(result_keywords, columns=['date', 'category', 'keyword', 'importance'])

result

,date,category,keyword,importance
0,2022.09.13,사회,정보통신망법,0.8443
1,2022.09.13,사회,통신비밀보호법,0.8406
2,2022.09.13,사회,서울중앙지법,0.8355
3,2022.09.13,사회,사법연수원,0.8345
4,2022.09.13,사회,문학평론가,0.8315
...,...,...,...,...
975,2022.09.19,연예,홈페이지,0.7316
976,2022.09.19,연예,미니음반,0.7308
977,2022.09.19,연예,화장실,0.7302
978,2022.09.19,연예,목격담,0.7289


In [69]:
result.to_csv('./Data/keywords.csv', index=False)

In [70]:
import pymysql

In [72]:
keywords = pd.read_csv('./Data/keywords.csv')

keywords

,date,category,keyword,importance
0,2022.09.13,사회,정보통신망법,0.8443
1,2022.09.13,사회,통신비밀보호법,0.8406
2,2022.09.13,사회,서울중앙지법,0.8355
3,2022.09.13,사회,사법연수원,0.8345
4,2022.09.13,사회,문학평론가,0.8315
...,...,...,...,...
975,2022.09.19,연예,홈페이지,0.7316
976,2022.09.19,연예,미니음반,0.7308
977,2022.09.19,연예,화장실,0.7302
978,2022.09.19,연예,목격담,0.7289


In [73]:
sql_insert = 'INSERT INTO keywords (date, category, keyword, importance) VALUES (%s, %s, %s, %s)'

In [76]:
conn = pymysql.connect(host='localhost', user='root', password='1234', db='dins', charset='utf8')
cursor = conn.cursor()


for index, row in keywords.iterrows():
    cursor.execute(sql_insert, (row.date, row.category, row.keyword, row.importance))

conn.commit()
conn.close()

In [106]:
# keywords = model.extract_keywords(contents_nouns, keyphrase_ngram_range=(1,1), top_n=15, use_mmr=True)

# keywords